In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from statsmodels.stats.proportion import proportion_confint # Tính khoảng tin cậy
import sys
import os
import pandas as pd
import numpy as np

In [82]:
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))

# Lấy thư mục hiện tại
current_directory = os.getcwd()
folder_path1 = os.path.join(current_directory,'../../data/train_test')

In [83]:
x = pd.read_csv(f'{folder_path1}/xtrain_data.csv')
y = pd.read_csv(f'{folder_path1}/xtest_data.csv')
z = pd.read_csv(f'{folder_path1}/ytrain_data.csv')
t = pd.read_csv(f'{folder_path1}/ytest_data.csv')

x_train = x.iloc[:, 1:].values
x_test = y.iloc[:, 1:].values

y_train = z['drop_out'].values
y_test = t['drop_out'].values





In [84]:
# Chuyển đổi y_train thành nhãn phân loại.
y_train_class = np.where(y_train >= 0.5, 1, 0)
y_test_class = np.where(y_test >= 0.5, 1, 0)

In [85]:
LR_model = LogisticRegression()
LR = LR_model.fit(x_train, y_train_class)
y_pred = LR_model.predict(x_test)

In [86]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [87]:
# Đánh giá mô hình
accuracy = accuracy_score(y_test_class, y_pred)
conf_matrix = confusion_matrix(y_test_class, y_pred)
class_report = classification_report(y_test_class, y_pred)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


Accuracy: 0.5039370078740157
Confusion Matrix:
[[64  0]
 [63  0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        64
           1       0.00      0.00      0.00        63

    accuracy                           0.50       127
   macro avg       0.25      0.50      0.34       127
weighted avg       0.25      0.50      0.34       127



c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [88]:
from sklearn.dummy import DummyClassifier # mô hình cơ sỏ
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_train, y_train)
baseline_prediction = dummy_clf.predict(x_test)
baseline_accuracy = accuracy_score(y_test_class, np.where(baseline_prediction==np.unique(baseline_prediction)[0],0,baseline_prediction))

In [89]:
print("Model accuracy:", accuracy)
print("Baseline accuracy:", baseline_accuracy)

Model accuracy: 0.5039370078740157
Baseline accuracy: 0.5039370078740157


In [90]:
[[64  0]
 [63  0]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2409815907.py, line 1)

In [91]:
# Độ chính xác toàn thể: (TN+TP)/N
overall_accuracy = round((64+0)/x_test.shape[0] * 100,2)
print("Độ chính xác toàn thể: ", overall_accuracy)
# Sai số toàn thể
overall_mistake = round((0+63)/x_test.shape[0] * 100,2)
print("Sai số toàn thể: ", overall_mistake)

# Độ nhạy - % dự đoán chính xác sinh viên thôi học: TP/(TP+FN)
# sensitivity = (0/(1+0)) * 100
print("Độ nhạy: ", 0)

# Sai số của dự đoán sinh viên thôi TP/(TP+FN)
mistake = 100 - 100
print("Tỉ lệ mô hình dự đoán sai sinh viên nghỉ học: ", mistake)
 
# Độ đặc hiệu (specificity), % dự đoán sinh viên không thôi học: TN/(TN+FP)

specificity = (64/(64+0))*100
print("Độ đặc hiệu: ", specificity)
mistake_specificity = 100 - specificity
print("Sai số tỉ lệ mô hình dự đoán chính xác sinh viên không thôi học: ", mistake_specificity)

Độ chính xác toàn thể:  50.39
Sai số toàn thể:  49.61
Độ nhạy:  0
Tỉ lệ mô hình dự đoán sai sinh viên nghỉ học:  0
Độ đặc hiệu:  100.0
Sai số tỉ lệ mô hình dự đoán chính xác sinh viên không thôi học:  0.0


In [ ]:
student_id = y.iloc[:, 1]
df_pred = pd.concat([student_id,pd.DataFrame(y_test_class)], axis = 1).rename(columns = {0: 'drop_out'})
df_pred.rename(columns = {0: 'drop_out'}, inplace=True)

In [ ]:
# Kết quả dự đoán
df_pred[df_pred['drop_out']==1]

,student_id,drop_out
0,2011956334,1
1,2021595788,1
3,1954177532,1
4,2000001302,1
6,1983739846,1
...,...,...
116,2200003944,1
118,2000004424,1
121,2037245042,1
122,2114305523,1


In [ ]:
### Dự đoán đúng.

In [ ]:
lower_bound, upper_bound = proportion_confint(sum(y_pred), len(y_pred), alpha=0.05, method='wilson')

In [ ]:
print('lower_bound: ', lower_bound)
print('upper_bound: ', upper_bound)

lower_bound:  0.0
upper_bound:  0.029359645293763375


In [ ]:
# lower_bound và upper_bound thấp cho thấy khả năng cao tin tưởng dự đoán này.

In [ ]:
class_report = classification_report(y_test_class, y_pred, output_dict=True)
df_class_report_df = pd.DataFrame(class_report)
df_class_report_df.to_csv('../visualization/data/logistic_regr.csv')

c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Program Files\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
